In [1]:
import numpy as np
import torch
from ax.service.ax_client import AxClient, ObjectiveProperties
from ax.modelbridge.modelbridge_utils import extract_search_space_digest
from ax.models.torch_base import TorchOptConfig

In [2]:
# Setup Ax client
ax_client = AxClient()
ax_client.create_experiment(
    name="test1",
    parameters=[
        {
            "name": "param" + str(feature_idx),
            "type": "range",
            "bounds": [-1.0, 1.0],
            "value_type": "float",
        }
        for feature_idx in range(10)
    ],
    objectives={
        "score": ObjectiveProperties(minimize=False),
    }
)

[INFO 02-12 17:38:50] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 02-12 17:38:50] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='param0', parameter_type=FLOAT, range=[-1.0, 1.0]), RangeParameter(name='param1', parameter_type=FLOAT, range=[-1.0, 1.0]), RangeParameter(name='param2', parameter_type=FLOAT, range=[-1.0, 1.0]), RangeParameter(name='param3', parameter_type=FLOAT, range=[-1.0, 1.0]), RangeParameter(name='param4', parameter_type=FLOAT, range=[-1.0, 1.0]), RangeParameter(name='param5', parameter_type=FLOAT, range=[-1.0, 1.0]), RangeParameter(name='param6', parameter_type=FLOAT, range=[-1.0, 1.0]), RangeParameter(name='param7', parameter_type=FLOAT, range=[-1.0, 1.0]), RangeParameter(name='param8', parameter_type=FLOAT, range=[-1.0, 1.0]), RangeParameter(name='param9',

In [3]:
# Initialize fixed random Gaussian centers and covariances
_centers = [
    np.random.randn(10) * 0.5,  # Center 1
    np.random.randn(10) * 0.5,  # Center 2
    np.random.randn(10) * 0.5   # Center 3
]

# Create fixed random positive definite covariance matrices
_covs = []
for _ in range(3):
    A = np.random.randn(10, 10)
    cov = A.T @ A + np.eye(10)  # Ensure positive definite
    _covs.append(cov * 0.3)  # Scale down to avoid too sharp peaks

def score(parameterization):
    # Convert parameterization to numpy array
    x = np.array([parameterization[f"param{i}"] for i in range(10)])
    
    # Sum up the Gaussians using fixed centers and covariances
    result = 0
    for center, cov in zip(_centers, _covs):
        diff = x - center
        result += np.exp(-0.5 * diff @ np.linalg.solve(cov, diff))
    
    return {"score": float(result)}


In [4]:
for _ in range(100):
    parameterization, trial_index = ax_client.get_next_trial()
    print(score(parameterization))
    ax_client.complete_trial(trial_index, raw_data=score(parameterization))

ax_client.get_best_parameters()

/Users/keenanpepper/ax-feature-importance/.venv/lib/python3.12/site-packages/ax/modelbridge/cross_validation.py:463: UserWarning: Encountered exception in computing model fit quality: RandomModelBridge does not support prediction.
  warn("Encountered exception in computing model fit quality: " + str(e))
[INFO 02-12 17:38:50] ax.service.ax_client: Generated new trial 0 with parameters {'param0': 0.694142, 'param1': -0.969714, 'param2': 0.140363, 'param3': -0.401517, 'param4': 0.678007, 'param5': -0.768378, 'param6': -0.589432, 'param7': -0.939017, 'param8': -0.676749, 'param9': -0.468304} using model Sobol.


[INFO 02-12 17:38:50] ax.service.ax_client: Completed trial 0 with data: {'score': (0.319113, None)}.
/Users/keenanpepper/ax-feature-importance/.venv/lib/python3.12/site-packages/ax/modelbridge/cross_validation.py:463: UserWarning: Encountered exception in computing model fit quality: RandomModelBridge does not support prediction.
  warn("Encountered exception in computing model fit quality: " + str(e))
[INFO 02-12 17:38:50] ax.service.ax_client: Generated new trial 1 with parameters {'param0': -0.246052, 'param1': 0.880806, 'param2': -0.087701, 'param3': 0.812069, 'param4': -0.575936, 'param5': 0.392971, 'param6': 0.412205, 'param7': 0.627377, 'param8': 0.14629, 'param9': 0.168723} using model Sobol.
[INFO 02-12 17:38:50] ax.service.ax_client: Completed trial 1 with data: {'score': (1.000659, None)}.
/Users/keenanpepper/ax-feature-importance/.venv/lib/python3.12/site-packages/ax/modelbridge/cross_validation.py:463: UserWarning: Encountered exception in computing model fit quality: Ran

{'score': 0.31911297624510215}
{'score': 1.0006592117841806}
{'score': 0.35525111085921207}
{'score': 1.031745509264005}
{'score': 0.813199754365072}
{'score': 1.0755933235071609}
{'score': 0.212931963769264}
{'score': 0.6324479651601468}
{'score': 0.11669713990765421}
{'score': 1.0028623331329856}
{'score': 0.10781785003608799}
{'score': 0.7366973478876977}
{'score': 0.3403163937636773}
{'score': 0.4523781429077542}
{'score': 0.3742443292656392}
{'score': 1.0119946230844628}
{'score': 0.3614870232762978}
{'score': 0.3469671976148677}
{'score': 0.7150039922418664}
{'score': 0.13969172558630413}


[INFO 02-12 17:38:51] ax.service.ax_client: Generated new trial 20 with parameters {'param0': -0.252779, 'param1': 0.836473, 'param2': -0.656343, 'param3': 0.305122, 'param4': -0.310077, 'param5': -0.087799, 'param6': 0.924093, 'param7': 0.029587, 'param8': -0.421283, 'param9': 0.137748} using model BoTorch.
[INFO 02-12 17:38:51] ax.service.ax_client: Completed trial 20 with data: {'score': (1.354156, None)}.


{'score': 1.3541560620433608}


...

[INFO 02-12 17:40:03] ax.service.ax_client: Generated new trial 98 with parameters {'param0': -0.029683, 'param1': 1.0, 'param2': -0.332549, 'param3': 0.341556, 'param4': 0.176793, 'param5': -1.0, 'param6': 1.0, 'param7': -1.0, 'param8': 0.988288, 'param9': -0.596119} using model BoTorch.
[INFO 02-12 17:40:03] ax.service.ax_client: Completed trial 98 with data: {'score': (0.58865, None)}.


{'score': 0.5886504089691492}


[INFO 02-12 17:40:04] ax.service.ax_client: Generated new trial 99 with parameters {'param0': -0.000958, 'param1': 0.543271, 'param2': 0.155297, 'param3': -0.073627, 'param4': -0.269632, 'param5': -0.332856, 'param6': 0.83546, 'param7': -0.28364, 'param8': -0.24538, 'param9': 0.205202} using model BoTorch.
[INFO 02-12 17:40:04] ax.service.ax_client: Completed trial 99 with data: {'score': (2.070621, None)}.


{'score': 2.0706212908948203}


({'param0': -0.010599285255149415,
  'param1': 0.4923290244000398,
  'param2': 0.15706890655117345,
  'param3': -0.10873100900468646,
  'param4': -0.23571248897480446,
  'param5': -0.27725269271440367,
  'param6': 0.803206859465347,
  'param7': -0.22849742887803648,
  'param8': -0.2410382181472478,
  'param9': 0.1919857036767616},
 ({'score': np.float64(2.077854975321675)},
  {'score': {'score': np.float64(5.3124900566051465e-06)}}))

In [9]:
original_search_space = ax_client.experiment.search_space.clone()
search_space = original_search_space.clone()
for t in ax_client.generation_strategy.model.transforms.values():
    search_space = t.transform_search_space(search_space)
print(original_search_space)
print(search_space)

SearchSpace(parameters=[RangeParameter(name='param0', parameter_type=FLOAT, range=[-1.0, 1.0]), RangeParameter(name='param1', parameter_type=FLOAT, range=[-1.0, 1.0]), RangeParameter(name='param2', parameter_type=FLOAT, range=[-1.0, 1.0]), RangeParameter(name='param3', parameter_type=FLOAT, range=[-1.0, 1.0]), RangeParameter(name='param4', parameter_type=FLOAT, range=[-1.0, 1.0]), RangeParameter(name='param5', parameter_type=FLOAT, range=[-1.0, 1.0]), RangeParameter(name='param6', parameter_type=FLOAT, range=[-1.0, 1.0]), RangeParameter(name='param7', parameter_type=FLOAT, range=[-1.0, 1.0]), RangeParameter(name='param8', parameter_type=FLOAT, range=[-1.0, 1.0]), RangeParameter(name='param9', parameter_type=FLOAT, range=[-1.0, 1.0])], parameter_constraints=[])
SearchSpace(parameters=[RangeParameter(name='param0', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='param1', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='param2', parameter_type=FLOAT, range=[0.0,

In [10]:
ax_client.generation_strategy.model.transforms

OrderedDict([('Cast', <ax.modelbridge.transforms.cast.Cast at 0x1538e9400>),
             ('Winsorize',
              <ax.modelbridge.transforms.winsorize.Winsorize at 0x157630d40>),
             ('RemoveFixed',
              <ax.modelbridge.transforms.remove_fixed.RemoveFixed at 0x14f676ea0>),
             ('OrderedChoiceToIntegerRange',
              <ax.modelbridge.transforms.choice_encode.OrderedChoiceToIntegerRange at 0x157632570>),
             ('OneHot',
              <ax.modelbridge.transforms.one_hot.OneHot at 0x1574f8ef0>),
             ('IntToFloat',
              <ax.modelbridge.transforms.int_to_float.IntToFloat at 0x1574f8e60>),
             ('Log', <ax.modelbridge.transforms.log.Log at 0x15750d010>),
             ('Logit', <ax.modelbridge.transforms.logit.Logit at 0x1538853a0>),
             ('UnitX',
              <ax.modelbridge.transforms.unit_x.UnitX at 0x1575b41d0>),
             ('IVW', <ax.modelbridge.transforms.ivw.IVW at 0x1538846b0>),
             ('Derelativiz

In [11]:
ax_client.generation_strategy

GenerationStrategy(name='Sobol+BoTorch', steps=[Sobol for 20 trials, BoTorch for subsequent trials])

In [13]:
ssd = extract_search_space_digest(search_space, list(search_space.parameters))

In [14]:
ssd

SearchSpaceDigest(feature_names=['param0', 'param1', 'param2', 'param3', 'param4', 'param5', 'param6', 'param7', 'param8', 'param9'], bounds=[(0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0)], ordinal_features=[], categorical_features=[], discrete_choices={}, task_features=[], fidelity_features=[], target_values={}, robust_digest=None)

In [15]:
opt_config = TorchOptConfig(torch.tensor([1.0]))

In [16]:
ax_client.generation_strategy.model.model.best_point(ssd, opt_config)

tensor([0.4947, 0.7462, 0.5785, 0.4456, 0.3821, 0.3614, 0.9016, 0.3858, 0.3795,
        0.5960], dtype=torch.float64)

In [18]:
ax_client.generation_strategy.model.model.best_point(ssd, opt_config) * 2 - 1

tensor([-0.0106,  0.4923,  0.1571, -0.1087, -0.2357, -0.2773,  0.8032, -0.2285,
        -0.2410,  0.1920], dtype=torch.float64)

In [17]:
ax_client.get_best_parameters()

({'param0': -0.010599285255149415,
  'param1': 0.4923290244000398,
  'param2': 0.15706890655117345,
  'param3': -0.10873100900468646,
  'param4': -0.23571248897480446,
  'param5': -0.27725269271440367,
  'param6': 0.803206859465347,
  'param7': -0.22849742887803648,
  'param8': -0.2410382181472478,
  'param9': 0.1919857036767616},
 ({'score': np.float64(2.077854975321675)},
  {'score': {'score': np.float64(5.3124900566051465e-06)}}))

In [32]:
opt_config = TorchOptConfig(torch.tensor([1.0]), fixed_features={0: 0.4947})

In [33]:
ax_client.generation_strategy.model.model.best_point(ssd, opt_config)

In [34]:
from botorch.acquisition import PosteriorMean
from botorch.optim import optimize_acqf
from torch import Tensor

def get_model_optimum(
    model,
    bounds: Tensor,
    fixed_features: dict[int, float],
    maximize: bool = True,
) -> Tensor:
    """Find point that maximizes/minimizes posterior mean subject to fixed feature constraints."""
    
    acq_function = PosteriorMean(model, maximize=maximize)
    
    optimal_point, _ = optimize_acqf(
        acq_function=acq_function,
        bounds=bounds,
        q=1,  # Single point
        num_restarts=5,  # Number of random restarts for optimization
        raw_samples=20,  # Number of samples for initialization
        fixed_features=fixed_features,
        return_best_only=True,
    )
    
    return optimal_point.squeeze(0)  # Remove batch dimension

In [35]:
get_model_optimum(ax_client.generation_strategy.model.model, torch.tensor([-1.0, 1.0]), {})

AttributeError: 'BoTorchModel' object has no attribute 'num_outputs'

In [36]:
def get_model_optimum(
    ax_model,  # This is the BoTorchModel from Ax
    bounds: Tensor,
    fixed_features: dict[int, float],
    maximize: bool = True,
) -> Tensor:
    """Find point that maximizes/minimizes posterior mean subject to fixed feature constraints."""
    
    # Get the underlying BoTorch model
    botorch_model = ax_model.model
    
    acq_function = PosteriorMean(botorch_model, maximize=maximize)
    
    optimal_point, _ = optimize_acqf(
        acq_function=acq_function,
        bounds=bounds,
        q=1,
        num_restarts=5,
        raw_samples=20,
        fixed_features=fixed_features,
        return_best_only=True,
    )
    
    return optimal_point.squeeze(0)

In [37]:
model = ax_client.generation_strategy.model.model  # Get the BoTorchModel
bounds = torch.stack([torch.zeros(10), torch.ones(10)])  # Create bounds tensor
optimum = get_model_optimum(model, bounds, fixed_features={0: 0.5})

AttributeError: 'BoTorchModel' object has no attribute 'model'

In [38]:
def get_model_optimum(
    ax_model,  # This is the BoTorchModel from Ax
    bounds: Tensor,
    fixed_features: dict[int, float],
    maximize: bool = True,
) -> Tensor:
    """Find point that maximizes/minimizes posterior mean subject to fixed feature constraints."""
    
    # Get the underlying BoTorch model through the surrogate
    botorch_model = ax_model.surrogate.model
    
    acq_function = PosteriorMean(botorch_model, maximize=maximize)
    
    optimal_point, _ = optimize_acqf(
        acq_function=acq_function,
        bounds=bounds,
        q=1,
        num_restarts=5,
        raw_samples=20,
        fixed_features=fixed_features,
        return_best_only=True,
    )
    
    return optimal_point.squeeze(0)

In [47]:
model = ax_client.generation_strategy.model.model  # Get the BoTorchModel
bounds = torch.stack([torch.zeros(10), torch.ones(10)])  # Create bounds tensor
optimum = get_model_optimum(model, bounds, fixed_features={0: 0.495})

In [48]:
optimum * 2 - 1

tensor([-0.0100,  0.5084,  0.1627, -0.1053, -0.2327, -0.2652,  0.8001, -0.2174,
        -0.2423,  0.2163])

In [ ]:
ax_client.get_best_parameters()

({'param0': -0.010599285255149415,
  'param1': 0.4923290244000398,
  'param2': 0.15706890655117345,
  'param3': -0.10873100900468646,
  'param4': -0.23571248897480446,
  'param5': -0.27725269271440367,
  'param6': 0.803206859465347,
  'param7': -0.22849742887803648,
  'param8': -0.2410382181472478,
  'param9': 0.1919857036767616},
 ({'score': np.float64(2.077854975321675)},
  {'score': {'score': np.float64(5.3124900566051465e-06)}}))